# Sentiment Classification

## Loading the dataset

In [1]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


In [2]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

In [3]:
from keras.preprocessing import sequence
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test =  sequence.pad_sequences(x_test, maxlen=maxlen)

In [4]:
import numpy as np
print("Train Labels:", np.unique(y_train))
print("Test Labels:", np.unique(y_test))

Train Labels: [0 1]
Test Labels: [0 1]


In [5]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    1,   14,   22,   16,   43,  530,
        973, 1622, 1385,   65,  458, 4468,   66, 3941,    4,  173,   36,
        256,    5,   25,  100,   43,  838,  112,   50,  670,    2,    9,
         35,  480,  284,    5,  150,    4,  172,  112,  167,    2,  336,
        385,   39,    4,  172, 4536, 1111,   17,  546,   38,   13,  447,
          4,  192,   50,   16,    6,  147, 2025,   19,   14,   22,    4,
       1920, 4613,  469,    4,   22,   71,   87,   

In [6]:
y_train[0]

1

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [7]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout 
from tensorflow.keras.layers import Bidirectional, Lambda, Flatten, Input, Add
from tensorflow.keras import backend as K
import tensorflow as tf

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=maxlen))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 20)           200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               96800     
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 297,001
Trainable params: 297,001
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 56s 2ms/sample - loss: 0.4814 - accuracy: 0.7655 - val_loss: 0.3958 - val_accuracy: 0.8208
Epoch 2/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.3220 - accuracy: 0.8681 - val_loss: 0.3390 - val_accuracy: 0.8591
Epoch 3/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.2272 - accuracy: 0.9130 - val_loss: 0.3248 - val_accuracy: 0.8703


In [10]:
model2 = Sequential()
model2.add(Embedding(vocab_size, 20, input_length=maxlen))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dropout(0.25))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 20)           200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               96800     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 297,001
Trainable params: 297,001
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model2.fit(x_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.5234 - accuracy: 0.7304 - val_loss: 0.3612 - val_accuracy: 0.8473
Epoch 2/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.2956 - accuracy: 0.8806 - val_loss: 0.3271 - val_accuracy: 0.8605
Epoch 3/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.2284 - accuracy: 0.9133 - val_loss: 0.3608 - val_accuracy: 0.8581


In [12]:
model3 = Sequential()
model3.add(Embedding(vocab_size, 20, input_length=maxlen))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.50))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 20)           200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               96800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 297,001
Trainable params: 297,001
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model3.fit(x_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 55s 2ms/sample - loss: 0.4563 - accuracy: 0.7820 - val_loss: 0.3842 - val_accuracy: 0.8399
Epoch 2/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.2897 - accuracy: 0.8883 - val_loss: 0.3337 - val_accuracy: 0.8593
Epoch 3/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.2372 - accuracy: 0.9115 - val_loss: 0.3318 - val_accuracy: 0.8691


In [14]:
model4 = Sequential()
model4.add(Embedding(vocab_size, 20, input_length=maxlen))
model4.add(LSTM(100))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 20)           200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 248,501
Trainable params: 248,501
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model4.fit(x_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 31s 1ms/sample - loss: 0.4618 - accuracy: 0.7716 - val_loss: 0.3428 - val_accuracy: 0.8578
Epoch 2/3
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.2855 - accuracy: 0.8887 - val_loss: 0.3585 - val_accuracy: 0.8484
Epoch 3/3
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.2076 - accuracy: 0.9226 - val_loss: 0.3201 - val_accuracy: 0.8648


In [16]:
model5 = Sequential()
model5.add(Embedding(vocab_size, 20, input_length=maxlen))
model5.add(LSTM(100))
model5.add(Dropout(0.5))
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model5.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 20)           200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 248,501
Trainable params: 248,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
model5.fit(x_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.4502 - accuracy: 0.7806 - val_loss: 0.3292 - val_accuracy: 0.8624
Epoch 2/3
25000/25000 [==============================] - 29s 1ms/sample - loss: 0.4595 - accuracy: 0.7764 - val_loss: 0.3979 - val_accuracy: 0.8354
Epoch 3/3
25000/25000 [==============================] - 29s 1ms/sample - loss: 0.3116 - accuracy: 0.8773 - val_loss: 0.3341 - val_accuracy: 0.8634


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [29]:
from keras import backend as K
inp = model.input[100]                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp, K.learning_phase()], outputs)]    # evaluation functions

In [30]:
outputs

[<tf.Tensor 'embedding/Identity:0' shape=(None, 300, 20) dtype=float32>,
 <tf.Tensor 'bidirectional/Identity:0' shape=(None, 200) dtype=float32>,
 <tf.Tensor 'dense/Identity:0' shape=(None, 1) dtype=float32>]

In [31]:
functors

In [18]:
scores = model.evaluate(x_test, y_test, verbose=0)

In [19]:
scores

[0.33181426030158995, 0.86908]

In [20]:
predictions = model.predict(x_test)

In [21]:
print("Printing few predictions:")
print("Actual Label:", y_test[100])
print("Predicted Label:", predictions[100])
print("Actual Label:", y_test[10])
print("Predicted Label:", predictions[10])
print("Actual Label:", y_test[9999])
print("Predicted Label:", predictions[9999])

Printing few predictions:
Actual Label: 1
Predicted Label: [0.0206005]
Actual Label: 1
Predicted Label: [0.82928574]
Actual Label: 0
Predicted Label: [0.0839752]
